## PREDICCIÓN DE TEXTO

In [1]:
import cohere
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")
if not cohere_api_key:
    raise ValueError("No API key found. Please set the COHERE_API_KEY environment variable.")

co = cohere.Client(cohere_api_key)

try:
    response = co.generate(
        model='command-xlarge-nightly',  
        prompt="Continúa la siguiente frase de manera creativa y fluida: "+
            "Un día, soy de tierra, me conecto con la realidad desde el "+
            "mantenimiento del orden y la rutina. En otro tiempo, muy ocasional, "+
            "soy aire, me vuelvo a encontrar con...",
        max_tokens=80,
        temperature=0.9
    )

    print(response.generations[0].text)

except cohere.CohereAPIError as e:
    print(f"An error occurred: {e}")

...la libertad y la creatividad. Soy capaz de volar, de imaginar y de crear sin límites. Me dejo llevar por el viento, por las ideas y por los sueños que me transportan a mundos nuevos y emocionantes. Pero, aunque soy aire, no me olvido de mis raíces, de mi conexión con la tierra, pues es allí donde encuentro la estabilidad y la fuerza para


## Plantillas de prompt

In [11]:
templates = {
   "historia": ("Continúa la siguiente historia de manera creativa y fluida: "
                 "Había una vez un {character} en un {place}, "
                 "que estaba {activity}. Un día, {event}. "
                 "{question}")
}

selected_template = templates["historia"]
prompt = selected_template.format(character="bravo trabajador", 
                                  place="país distópico", 
                                  activity="rompiéndose el lomo", 
                                  event="descubrió que estaba siendo estafado",
                                  question = "¿Tiene final feliz?")


response = co.generate(
        model='command-xlarge-nightly',  
        prompt=prompt,
        max_tokens=80,
        temperature=0.7
    )
print(response.generations[0].text)

Había una vez un bravo trabajador en un país distópico que, día tras día, se rompía el lomo para salir adelante. Su trabajo era arduo y agotador, pero lo realizaba con dedicación y orgullo. Sin embargo, un día, descubrió una verdad impactante: estaba siendo estafado por su propio gobierno.

El trabajador se dio cuenta de que


### Parámetros (temperatura, k y p)

In [41]:
response = co.generate(
        model='command-xlarge-nightly',  
        prompt='Name a country from Oceania',
        max_tokens=5,
        temperature=0.1,
        k=5,
        p=0.9
    )
print(response.generations[0].text)

New Zealand is a country


## LangSmith

In [46]:
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r-plus")

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="goodbye!"),
]

model.invoke(messages)

AIMessage(content='Arrivederci!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7a76c2c8-d776-4192-ae24-0ebb1dbd1354', 'token_count': {'input_tokens': 79, 'output_tokens': 4}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '7a76c2c8-d776-4192-ae24-0ebb1dbd1354', 'token_count': {'input_tokens': 79, 'output_tokens': 4}}, id='run-11c6a5a2-d7cd-4287-8ff2-acdfa545e02c-0', usage_metadata={'input_tokens': 79, 'output_tokens': 4, 'total_tokens': 83})

In [47]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

'Arrivederci!'

### prompt-templates

In [48]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "french", "text": "cigüeña"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french:'), HumanMessage(content='cigüeña')])

In [49]:
chain = prompt_template | model | parser

chain.invoke({"language": "french", "text": "cigüeña"})

'La traduction de "cigüeña" en français est "cigogne".'